In [1]:
import os
import pytesseract
from docx import Document
from pdf2image import convert_from_path
import nltk
import pandas as pd
import re

In [2]:
# Download NLTK resources
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\velam\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
from pdf2image import convert_from_path

# Use your full Poppler bin path
poppler_path = r"C:\Release-24.08.0-0\poppler-24.08.0\Library\bin"
images = convert_from_path(r"C:\amrita_uni\Projects\asmacs internship\recognition of skills and title\Imgdownload2.pdf", poppler_path=poppler_path)

# Just to confirm
for i, img in enumerate(images):
    img.save(f"page_{i+1}.png")


In [1]:
# Set up your paths correctly before running
tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
poppler_path = r"C:\Release-24.08.0-0\poppler-24.08.0\Library\bin"
pytesseract.pytesseract.tesseract_cmd = tesseract_cmd

NameError: name 'pytesseract' is not defined

In [8]:
def extract_text_from_pdf(pdf_path):
    print("[INFO] Converting PDF pages to images...")
    images = convert_from_path(pdf_path, poppler_path=poppler_path)
    text = ""
    for i, img in enumerate(images):
        print(f"[INFO] Running OCR on page {i+1}")
        text += pytesseract.image_to_string(img)
    return text

def extract_text_from_docx(docx_path):
    print("[INFO] Reading DOCX file...")
    doc = Document(docx_path)
    return "\n".join([para.text for para in doc.paragraphs])

def load_keywords_from_folder(folder_path):
    keywords = set()
    for filename in os.listdir(folder_path):
        if filename.endswith(".txt") or filename.endswith(".csv"):
            with open(os.path.join(folder_path, filename), encoding='utf-8') as f:
                for line in f:
                    for kw in line.strip().split(","):
                        if kw.strip():
                            keywords.add(kw.strip().lower())
    return list(keywords)

def extract_keywords(text, keywords):
    found = []
    for kw in keywords:
        if re.search(r'\b' + re.escape(kw) + r'\b', text.lower()):
            found.append(kw)
    return found

def process_resume(file_path, skills_folder, titles_folder):
    if file_path.endswith(".pdf"):
        text = extract_text_from_pdf(file_path)
    elif file_path.endswith(".docx"):
        text = extract_text_from_docx(file_path)
    else:
        raise ValueError("Unsupported file type. Please use PDF or DOCX.")

    print("[INFO] Loading skills and job titles...")
    skills = load_keywords_from_folder(skills_folder)
    titles = load_keywords_from_folder(titles_folder)

    print("[INFO] Extracting skills and titles from resume text...")
    found_skills = extract_keywords(text, skills)
    found_titles = extract_keywords(text, titles)

    print("\n===== RESUME PARSE RESULT =====")
    print("Top 500 Characters of Extracted Text:\n", text[:500], "\n")
    print("Skills Found:", found_skills)
    print("Job Titles Found:", found_titles)

    return {
        "text": text,
        "skills": found_skills,
        "titles": found_titles
    }

# === EXAMPLE USAGE ===
if __name__ == "__main__":
    file_path = r"C:\amrita_uni\Projects\asmacs internship\recognition of skills and title\Imgdownload2.pdf"  # <-- Replace with your file
    skills_folder = r"C:\amrita_uni\Projects\asmacs internship\recognition of skills and title\unique_job_skills_parts"  # Folder with .txt/.csv skills
    titles_folder = r"C:\amrita_uni\Projects\asmacs internship\recognition of skills and title\unique_job_titles_parts"  # Folder with .txt/.csv titles

    result = process_resume(file_path, skills_folder, titles_folder)


[INFO] Converting PDF pages to images...
[INFO] Running OCR on page 1
[INFO] Running OCR on page 2
[INFO] Running OCR on page 3
[INFO] Running OCR on page 4
[INFO] Loading skills and job titles...
[INFO] Extracting skills and titles from resume text...

===== RESUME PARSE RESULT =====
Top 500 Characters of Extracted Text:
 DIGVIJAYSINGHPARMAR

Permanent Address: 40/C, Sangam Nagar, Indore, M.P — 452006
Present Address:2195 NiralaKunj, Gas AthourityOf India Township
Dibiyapur, Auraiya, Uttar Pradesh, |ndia-206244

Emait:digv10sp@gmail.com, digvijay10sp@outlook.com
Mobile: +91-9662489663

in ee Jinkedin.com/in/digvijay-singh-parmar-aabb0953

CAREER OBJECTIVE

To utilize my ability and knowledge about Safety and Emergency control methods, including
the detection of risks to prevent life-threatening harms and damages  

Skills Found: ['mobilization', 'testing', 'control system', 'on', '18001', 'marital status', 'undertaking', 'specific job', 'de', 'listening', 'medium', 'schedule', 'pt', '

(above)this is using re to match skills and titles

In [1]:
import torch

print("Torch Version:", torch.__version__)
print("CUDA Available:", torch.cuda.is_available())
print("CUDA Version (if available):", torch.version.cuda)
print("Device:", torch.device("cuda" if torch.cuda.is_available() else "cpu"))


Torch Version: 2.5.0+cu124
CUDA Available: True
CUDA Version (if available): 12.4
Device: cuda


below using spacy just some trail not using dataset

In [2]:
import spacy
import re

# Load spaCy's English model
nlp = spacy.load("en_core_web_sm")

In [4]:


# List of predefined job titles for comparison
job_titles_db = [
    'licensed assistant salon manager', 'store manager', 'nurse', 'software engineer', 'data analyst', 
    'project manager', 'sales manager', 'mechanical engineer', 'hr manager', 'safety officer', 
    'maintenance manager', 'senior engineer', 'research assistant', 'chief technology officer', 
    'civil engineer', 'product manager', 'marketing manager', 'accounting manager'
    # Add more job titles as required
]

# Example list of predefined skills (expand this list as needed)
skills_db = [
    'customer service', 'communication', 'problem solving', 'teamwork', 'leadership', 
    'project management', 'data analysis', 'marketing', 'strategic planning', 'coding', 'software development',
    'nursing', 'patient care', 'sales', 'negotiation', 'time management', 'critical thinking'
    # Add more skills as required
]

# Function to extract job titles using spaCy NER and custom rules
def extract_job_titles(text):
    # Process the text with spaCy
    doc = nlp(text)
    
    # Define the custom job title patterns (using tokens like 'manager', 'engineer', etc.)
    job_title_keywords = ['Manager', 'engineer', 'assistant', 'director', 'supervisor', 'specialist', 'officer']
    
    # Extract NER entities that might be job titles (check for relevant keywords)
    job_titles = []
    for ent in doc.ents:
        if ent.label_ in ['ORG', 'PERSON']:  # Consider organization names or people for titles
            if any(keyword in ent.text.lower() for keyword in job_title_keywords):
                job_titles.append(ent.text.strip())
    
    # Filter out duplicates and return
    return list(set(job_titles))

# Function to extract skills using regex based matching or predefined list
def extract_skills(text):
    # Process the text with spaCy
    doc = nlp(text)
    
    # Extract skills based on the predefined list
    extracted_skills = []
    for skill in skills_db:
        if re.search(r'\b' + re.escape(skill) + r'\b', text, re.IGNORECASE):
            extracted_skills.append(skill)
    
    # Filter out duplicates and return
    return list(set(extracted_skills))

# Sample text input
text_input = """
Licensed Assistant Salon Manager with 5 years of experience. I specialize in customer service, communication, 
teamwork, and problem solving. I have also worked as a Store Manager for 3 years. Experienced with sales, marketing, 
and project management. Previously, I worked as a Nurse, focusing on patient care and leadership in healthcare settings.
"""

# Extract job titles and skills from the input text
job_titles = extract_job_titles(text_input)
skills = extract_skills(text_input)

print("Extracted Job Titles:", job_titles)
print("Extracted Skills:", skills)


Extracted Job Titles: []
Extracted Skills: ['problem solving', 'project management', 'customer service', 'communication', 'marketing', 'teamwork', 'sales', 'patient care', 'leadership']


using spacy phrasematcher

In [1]:
import os
import re
import spacy
from spacy.matcher import PhraseMatcher
from pdf2image import convert_from_path
import pytesseract
from docx import Document

# Set your poppler path (update accordingly if needed)
poppler_path = r"C:\path\to\poppler\bin"  # Replace with your poppler path
tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
poppler_path = r"C:\Release-24.08.0-0\poppler-24.08.0\Library\bin"
pytesseract.pytesseract.tesseract_cmd = tesseract_cmd
# Load spaCy model
nlp = spacy.load("en_core_web_sm")

def extract_text_from_pdf(pdf_path):
    print("[INFO] Converting PDF pages to images...")
    images = convert_from_path(pdf_path, poppler_path=poppler_path)
    text = ""
    for i, img in enumerate(images):
        print(f"[INFO] Running OCR on page {i+1}")
        text += pytesseract.image_to_string(img)
    return text

def extract_text_from_docx(docx_path):
    print("[INFO] Reading DOCX file...")
    doc = Document(docx_path)
    return "\n".join([para.text for para in doc.paragraphs])

def load_keywords_from_folder(folder_path):
    keywords = set()
    for filename in os.listdir(folder_path):
        if filename.endswith(".txt") or filename.endswith(".csv"):
            with open(os.path.join(folder_path, filename), encoding='utf-8') as f:
                for line in f:
                    for kw in line.strip().split(","):
                        kw_clean = kw.strip()
                        if kw_clean:
                            keywords.add(kw_clean.lower())
    return list(keywords)

def create_phrase_matcher(keywords, label):
    matcher = PhraseMatcher(nlp.vocab, attr="LOWER")
    patterns = [nlp.make_doc(text) for text in keywords]
    matcher.add(label, patterns)
    return matcher

def extract_keywords_with_spacy(text, skills_keywords, titles_keywords):
    doc = nlp(text)
    skills_matcher = create_phrase_matcher(skills_keywords, "SKILL")
    titles_matcher = create_phrase_matcher(titles_keywords, "TITLE")

    skills_found = set()
    titles_found = set()

    for match_id, start, end in skills_matcher(doc):
        span = doc[start:end]
        skills_found.add(span.text)

    for match_id, start, end in titles_matcher(doc):
        span = doc[start:end]
        titles_found.add(span.text)

    return list(skills_found), list(titles_found)

def process_resume(file_path, skills_folder, titles_folder):
    if file_path.endswith(".pdf"):
        text = extract_text_from_pdf(file_path)
    elif file_path.endswith(".docx"):
        text = extract_text_from_docx(file_path)
    else:
        raise ValueError("Unsupported file type. Please use PDF or DOCX.")

    print("[INFO] Loading skills and job titles...")
    skills = load_keywords_from_folder(skills_folder)
    titles = load_keywords_from_folder(titles_folder)

    print("[INFO] Extracting skills and titles using spaCy...")
    found_skills, found_titles = extract_keywords_with_spacy(text, skills, titles)

    print("\n===== RESUME PARSE RESULT =====")
    print("Top 500 Characters of Extracted Text:\n", text[:500], "\n")
    print("Skills Found:", found_skills)
    print("Job Titles Found:", found_titles)

    return {
        "text": text,
        "skills": found_skills,
        "titles": found_titles
    }

# === EXAMPLE USAGE ===
if __name__ == "__main__":
    file_path = r"C:\Users\velam\OneDrive\Documents\pavan's resume.docx"
    skills_folder = r"C:\amrita_uni\Projects\asmacs internship\recognition of skills and title\unique_job_skills_parts"
    titles_folder = r"C:\amrita_uni\Projects\asmacs internship\recognition of skills and title\unique_job_titles_parts"

    result = process_resume(file_path, skills_folder, titles_folder)


[INFO] Reading DOCX file...
[INFO] Loading skills and job titles...
[INFO] Extracting skills and titles using spaCy...

===== RESUME PARSE RESULT =====
Top 500 Characters of Extracted Text:
 Velamala Pavan Krishna

SUMMARY
Passionate computer science and AI student with a strong foundation in machine learning, data analysis, and model building . Eager to explore emerging technologies and apply creative solutions to real-world problems . Committed to continuous learning, collaboration, and making a meaningful impact through AI.

Phone:
+91 7569637875
Email:
velamalapavankrishna@gmail.com
Address: Visakhapatnam , 
Andhra Pradesh , India
    530046

Profile links

Linkedin: linkedin.co 

Skills Found: ['and', 'strong foundation', 'Language', 'IBM', 'C', 'symptoms', 'management system', 'emerging technologies', 'Collaboration', 'technologies', 'HealthCare AI', 'HealthCare', 'Database', 'model', 'computer', 'Tools', 'Data Structures', 'making', 'Mar', 'Notebook', 'Azure', 'reservation', 'ML

this is using spacy phrasematcher and lemmetizing skills and titles ( dont use it)

In [2]:
import os
import re
import pytesseract
import spacy
from spacy.matcher import PhraseMatcher
from pdf2image import convert_from_path
from docx import Document
from PIL import Image
import tempfile

# Load spaCy English model
nlp = spacy.load("en_core_web_sm")

# Set your poppler path (update accordingly if needed)
tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
poppler_path = r"C:\Release-24.08.0-0\poppler-24.08.0\Library\bin"
pytesseract.pytesseract.tesseract_cmd = tesseract_cmd


def extract_text_from_pdf(pdf_path):
    print("[INFO] Converting PDF pages to images...")
    images = convert_from_path(pdf_path, poppler_path=poppler_path)
    text = ""
    for i, img in enumerate(images):
        print(f"[INFO] Running OCR on page {i+1}")
        text += pytesseract.image_to_string(img)
    return text

def extract_text_from_docx(docx_path):
    print("[INFO] Reading DOCX file...")
    doc = Document(docx_path)
    return "\n".join([para.text for para in doc.paragraphs])

# --- FILE LOADER (Skills/Titles) ---

def load_terms_from_folder(folder_path):
    terms = set()
    for file in os.listdir(folder_path):
        if file.endswith('.txt') or file.endswith('.csv'):
            with open(os.path.join(folder_path, file), 'r', encoding='utf-8') as f:
                for line in f:
                    items = re.split(r',|;', line.strip())
                    for item in items:
                        cleaned = item.strip().lower()
                        if cleaned:
                            terms.add(cleaned)
    return terms

def lemmatize_phrase(phrase):
    doc = nlp(phrase.lower())
    return ' '.join([token.lemma_ for token in doc if token.is_alpha])

# --- MATCHER SETUP ---

def build_matcher(nlp, terms):
    matcher = PhraseMatcher(nlp.vocab, attr='LOWER')
    patterns = [nlp.make_doc(term) for term in terms]
    matcher.add("TERM_MATCH", patterns)
    return matcher

def extract_matched_terms(text, matcher):
    doc = nlp(text)
    matches = matcher(doc)
    matched = set()
    for match_id, start, end in matches:
        span = doc[start:end]
        matched.add(span.text.lower())
    return list(matched)

# --- MAIN PROCESS ---

def extract_skills_and_titles(file_path, skills_folder, titles_folder):
    ext = os.path.splitext(file_path)[1].lower()
    if ext == ".pdf":
        text = extract_text_from_pdf(file_path)
    elif ext == ".docx":
        text = extract_text_from_docx(file_path)
    else:
        raise ValueError("Unsupported file type. Only PDF and DOCX supported.")

    # Preprocess
    print("Lemmatizing")
    lemmatized_text = lemmatize_phrase(text)

    # Load terms
    print("skill extraction...")
    raw_skills = load_terms_from_folder(skills_folder)
    print("titles extraction..")
    raw_titles = load_terms_from_folder(titles_folder)
    print("Lemmatizing skills...")
    lemmatized_skills = {lemmatize_phrase(term) for term in raw_skills}
    print("Lemmatizing titles..")
    lemmatized_titles = {lemmatize_phrase(term) for term in raw_titles}

    # Build matchers
    print("building matcher...")
    skills_matcher = build_matcher(nlp, lemmatized_skills)
    titles_matcher = build_matcher(nlp, lemmatized_titles)
    print("matching terms..")
    # Match terms
    matched_skills = extract_matched_terms(lemmatized_text, skills_matcher)
    matched_titles = extract_matched_terms(lemmatized_text, titles_matcher)

    return sorted(set(matched_skills)), sorted(set(matched_titles))

# --- EXAMPLE USAGE ---

if __name__ == "__main__":
    # Replace with your actual paths
    file_path = r"C:\amrita_uni\Projects\asmacs internship\recognition of skills and title\Imgdownload2.pdf"
    skills_folder = r"C:\amrita_uni\Projects\asmacs internship\recognition of skills and title\unique_job_skills_parts"
    titles_folder = r"C:\amrita_uni\Projects\asmacs internship\recognition of skills and title\unique_job_titles_parts"
 

    skills, titles = extract_skills_and_titles(file_path, skills_folder, titles_folder)

    print("Extracted Skills:")
    print(list(skills))

    print("\nExtracted Job Titles:")
    print(list(titles))

[INFO] Converting PDF pages to images...
[INFO] Running OCR on page 1
[INFO] Running OCR on page 2
[INFO] Running OCR on page 3
[INFO] Running OCR on page 4
Lemmatizing
skill extraction...
titles extraction..
Lemmatizing skills...


KeyboardInterrupt: 

dont embark on the path of lemmetinzing skills and titles, it took 500 mins and yet skills were too large to be lemmetized 

below is using same spacy code on new dataset 

In [3]:
import os
import re
import spacy
from spacy.matcher import PhraseMatcher
from pdf2image import convert_from_path
import pytesseract
from docx import Document

# Set your poppler path (update accordingly if needed)
tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
poppler_path = r"C:\Release-24.08.0-0\poppler-24.08.0\Library\bin"
pytesseract.pytesseract.tesseract_cmd = tesseract_cmd
# Load spaCy model
nlp = spacy.load("en_core_web_sm")

def extract_text_from_pdf(pdf_path):
    print("[INFO] Converting PDF pages to images...")
    images = convert_from_path(pdf_path, poppler_path=poppler_path)
    text = ""
    for i, img in enumerate(images):
        print(f"[INFO] Running OCR on page {i+1}")
        text += pytesseract.image_to_string(img)
    return text

def extract_text_from_docx(docx_path):
    print("[INFO] Reading DOCX file...")
    doc = Document(docx_path)
    return "\n".join([para.text for para in doc.paragraphs])
"""
def load_keywords_from_folder(folder_path):
    keywords = set()
    for filename in os.listdir(folder_path):
        if filename.endswith(".txt") or filename.endswith(".csv"):
            with open(os.path.join(folder_path, filename), encoding='utf-8') as f:
                for line in f:
                    for kw in line.strip().split(","):
                        kw_clean = kw.strip()
                        if kw_clean:
                            keywords.add(kw_clean.lower())
    return list(keywords)
"""
def load_keywords_from_folder(folder_path):
    keywords = set()
    for filename in os.listdir(folder_path):
        if filename.endswith(".txt") or filename.endswith(".csv"):
            with open(os.path.join(folder_path, filename), encoding='utf-8') as f:
                for line in f:
                    # Don't split on commas if you want multi-word phrases
                    kw_clean = line.strip().lower()
                    if kw_clean:
                        keywords.add(kw_clean)
    return list(keywords)

def create_phrase_matcher(keywords, label):
    matcher = PhraseMatcher(nlp.vocab, attr="LOWER")
    patterns = [nlp.make_doc(text) for text in keywords]
    matcher.add(label, patterns)
    return matcher

def extract_keywords_with_spacy(text, skills_keywords, titles_keywords):
    doc = nlp(text)
    skills_matcher = create_phrase_matcher(skills_keywords, "SKILL")
    titles_matcher = create_phrase_matcher(titles_keywords, "TITLE")

    skills_found = set()
    titles_found = set()

    for match_id, start, end in skills_matcher(doc):
        span = doc[start:end]
        skills_found.add(span.text)

    for match_id, start, end in titles_matcher(doc):
        span = doc[start:end]
        titles_found.add(span.text)

    return list(skills_found), list(titles_found)

def process_resume(file_path, skills_folder, titles_folder):
    if file_path.endswith(".pdf"):
        text = extract_text_from_pdf(file_path)
    elif file_path.endswith(".docx"):
        text = extract_text_from_docx(file_path)
    else:
        raise ValueError("Unsupported file type. Please use PDF or DOCX.")

    print("[INFO] Loading skills and job titles...")
    skills = load_keywords_from_folder(skills_folder)
    titles = load_keywords_from_folder(titles_folder)

    print("[INFO] Extracting skills and titles using spaCy...")
    found_skills, found_titles = extract_keywords_with_spacy(text, skills, titles)

    print("\n===== RESUME PARSE RESULT =====")
    print("Top 500 Characters of Extracted Text:\n", text[:500], "\n")
    print("Skills Found:", found_skills)
    print("Job Titles Found:", found_titles)

    return {
        "text": text,
        "skills": found_skills,
        "titles": found_titles
    }

# === EXAMPLE USAGE ===
if __name__ == "__main__":
    file_path = r"C:\Users\velam\OneDrive\Documents\pavan's resume.docx"
    skills_folder = r"C:\amrita_uni\Projects\asmacs internship\recognition of skills and title\skills"
    titles_folder = r"C:\amrita_uni\Projects\asmacs internship\recognition of skills and title\titles"
    file_path2= r"C:\amrita_uni\Projects\asmacs internship\recognition of skills and title\Imgdownload2.pdf"
    result = process_resume(file_path, skills_folder, titles_folder)
    result=process_resume(file_path2, skills_folder, titles_folder)

[INFO] Reading DOCX file...
[INFO] Loading skills and job titles...
[INFO] Extracting skills and titles using spaCy...

===== RESUME PARSE RESULT =====
Top 500 Characters of Extracted Text:
 Velamala Pavan Krishna

SUMMARY
Passionate computer science and AI student with a strong foundation in machine learning, data analysis, and model building . Eager to explore emerging technologies and apply creative solutions to real-world problems . Committed to continuous learning, collaboration, and making a meaningful impact through AI.

Phone:
+91 7569637875
Email:
velamalapavankrishna@gmail.com
Address: Visakhapatnam , 
Andhra Pradesh , India
    530046

Profile links

Linkedin: linkedin.co 

Skills Found: ['collaboration', 'Matlab', 'project', 'management system', 'Collaboration', 'Algorithms', 'Adaptive learning', 'Web Development', 'Developers', 'data analysis', 'Azure', 'RISK', 'html', 'scratch', 'Microsoft', 'Data Science', 'CS', 'building', 'Java', 'Fundamentals', 'HealthCare', 'ML', 'C'

spacy ner model with out custom model trail on skills and titles for above code

In [ ]:
import os
import spacy
from spacy.matcher import PhraseMatcher
from pdf2image import convert_from_path
import pytesseract
from docx import Document

# Set your poppler and tesseract paths
tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
poppler_path = r"C:\Release-24.08.0-0\poppler-24.08.0\Library\bin"
pytesseract.pytesseract.tesseract_cmd = tesseract_cmd

# Load spaCy model
nlp = spacy.load("en_core_web_sm")  

def extract_text_from_pdf(pdf_path):
    print("[INFO] Converting PDF pages to images...")
    images = convert_from_path(pdf_path, poppler_path=poppler_path)
    text = ""
    for i, img in enumerate(images):
        print(f"[INFO] Running OCR on page {i+1}")
        text += pytesseract.image_to_string(img)
    return text

def extract_text_from_docx(docx_path):
    print("[INFO] Reading DOCX file...")
    doc = Document(docx_path)
    return "\n".join([para.text for para in doc.paragraphs])

def load_keywords_from_folder(folder_path):
    keywords = set()
    for filename in os.listdir(folder_path):
        if filename.endswith(".txt") or filename.endswith(".csv"):
            with open(os.path.join(folder_path, filename), encoding='utf-8') as f:
                for line in f:
                    kw_clean = line.strip().lower()
                    if kw_clean:
                        keywords.add(kw_clean)
    return list(keywords)

def create_phrase_matcher(keywords, label):
    matcher = PhraseMatcher(nlp.vocab, attr="LOWER")
    patterns = [nlp.make_doc(text) for text in keywords]
    matcher.add(label, patterns)
    return matcher

def extract_keywords_with_spacy(text, skills_keywords, titles_keywords):
    doc = nlp(text)
    skills_matcher = create_phrase_matcher(skills_keywords, "SKILL")
    titles_matcher = create_phrase_matcher(titles_keywords, "TITLE")

    skills_found = set()
    titles_found = set()

    # PhraseMatcher results
    for match_id, start, end in skills_matcher(doc):
        span = doc[start:end]
        skills_found.add(span.text)

    for match_id, start, end in titles_matcher(doc):
        span = doc[start:end]
        titles_found.add(span.text)

    # NER results
    for ent in doc.ents:
        if ent.label_.lower() in {"skill", "job_title", "work_of_art", "org", "product"}:
            # Add additional logic if needed to separate skills from titles
            if "developer" in ent.text.lower() or "engineer" in ent.text.lower():
                titles_found.add(ent.text)
            else:
                skills_found.add(ent.text)

    return list(skills_found), list(titles_found)

def process_resume(file_path, skills_folder, titles_folder):
    if file_path.endswith(".pdf"):
        text = extract_text_from_pdf(file_path)
    elif file_path.endswith(".docx"):
        text = extract_text_from_docx(file_path)
    else:
        raise ValueError("Unsupported file type. Please use PDF or DOCX.")

    print("[INFO] Loading skills and job titles...")
    skills = load_keywords_from_folder(skills_folder)
    titles = load_keywords_from_folder(titles_folder)

    print("[INFO] Extracting skills and titles using spaCy...")
    found_skills, found_titles = extract_keywords_with_spacy(text, skills, titles)

    print("\n===== RESUME PARSE RESULT =====")
    print("Top 500 Characters of Extracted Text:\n", text[:500], "\n")
    print("Skills Found:", found_skills)
    print("Job Titles Found:", found_titles)

    return {
        "text": text,
        "skills": found_skills,
        "titles": found_titles
    }

if __name__ == "__main__":
    file_path = r"C:\Users\velam\OneDrive\Documents\pavan's resume.docx"
    skills_folder = r"C:\amrita_uni\Projects\asmacs internship\recognition of skills and title\skills"
    titles_folder = r"C:\amrita_uni\Projects\asmacs internship\recognition of skills and title\titles"
    file_path2 = r"C:\amrita_uni\Projects\asmacs internship\recognition of skills and title\Imgdownload2.pdf"
    
    result1 = process_resume(file_path, skills_folder, titles_folder)
    result2 = process_resume(file_path2, skills_folder, titles_folder)


[INFO] Reading DOCX file...
[INFO] Loading skills and job titles...
[INFO] Extracting skills and titles using spaCy...

===== RESUME PARSE RESULT =====
Top 500 Characters of Extracted Text:
 Velamala Pavan Krishna

SUMMARY
Passionate computer science and AI student with a strong foundation in machine learning, data analysis, and model building . Eager to explore emerging technologies and apply creative solutions to real-world problems . Committed to continuous learning, collaboration, and making a meaningful impact through AI.

Phone:
+91 7569637875
Email:
velamalapavankrishna@gmail.com
Address: Visakhapatnam , 
Andhra Pradesh , India
    530046

Profile links

Linkedin: linkedin.co 

Skills Found: ['Library management', 'html', 'Microsoft', 'py', 'Data Science', 'Web Development', 'Data Structures', 'Algorithms', 'C', 'VScode', 'data analysis', 'IBM', 'ICTR-3 (INTERNATIONAL CONFERENCE ON TSUNAMI RISK REDUCTION& RESILIENCE', 'AI Fundamentals', 'collaboration', 'MySQL', 'scratch', 'mach

lets do the same but with other dataset (given one)

In [1]:
import os
import spacy
from spacy.matcher import PhraseMatcher
from pdf2image import convert_from_path
import pytesseract
from docx import Document

# Set your poppler and tesseract paths
tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
poppler_path = r"C:\Release-24.08.0-0\poppler-24.08.0\Library\bin"
pytesseract.pytesseract.tesseract_cmd = tesseract_cmd

# Load spaCy model
nlp = spacy.load("en_core_web_sm")  

def extract_text_from_pdf(pdf_path):
    print("[INFO] Converting PDF pages to images...")
    images = convert_from_path(pdf_path, poppler_path=poppler_path)
    text = ""
    for i, img in enumerate(images):
        print(f"[INFO] Running OCR on page {i+1}")
        text += pytesseract.image_to_string(img)
    return text

def extract_text_from_docx(docx_path):
    print("[INFO] Reading DOCX file...")
    doc = Document(docx_path)
    return "\n".join([para.text for para in doc.paragraphs])

def load_keywords_from_folder(folder_path):
    keywords = set()
    for filename in os.listdir(folder_path):
        if filename.endswith(".txt") or filename.endswith(".csv"):
            with open(os.path.join(folder_path, filename), encoding='utf-8') as f:
                for line in f:
                    kw_clean = line.strip().lower()
                    if kw_clean:
                        keywords.add(kw_clean)
    return list(keywords)

def create_phrase_matcher(keywords, label):
    matcher = PhraseMatcher(nlp.vocab, attr="LOWER")
    patterns = [nlp.make_doc(text) for text in keywords]
    matcher.add(label, patterns)
    return matcher

def extract_keywords_with_spacy(text, skills_keywords, titles_keywords):
    doc = nlp(text)
    skills_matcher = create_phrase_matcher(skills_keywords, "SKILL")
    titles_matcher = create_phrase_matcher(titles_keywords, "TITLE")

    skills_found = set()
    titles_found = set()

    # PhraseMatcher results
    for match_id, start, end in skills_matcher(doc):
        span = doc[start:end]
        skills_found.add(span.text)

    for match_id, start, end in titles_matcher(doc):
        span = doc[start:end]
        titles_found.add(span.text)

    # NER results
    for ent in doc.ents:
        if ent.label_.lower() in {"skill", "job_title", "work_of_art", "org", "product"}:
            # Add additional logic if needed to separate skills from titles
            if "developer" in ent.text.lower() or "engineer" in ent.text.lower():
                titles_found.add(ent.text)
            else:
                skills_found.add(ent.text)

    return list(skills_found), list(titles_found)

def process_resume(file_path, skills_folder, titles_folder):
    if file_path.endswith(".pdf"):
        text = extract_text_from_pdf(file_path)
    elif file_path.endswith(".docx"):
        text = extract_text_from_docx(file_path)
    else:
        raise ValueError("Unsupported file type. Please use PDF or DOCX.")

    print("[INFO] Loading skills and job titles...")
    skills = load_keywords_from_folder(skills_folder)
    titles = load_keywords_from_folder(titles_folder)

    print("[INFO] Extracting skills and titles using spaCy...")
    found_skills, found_titles = extract_keywords_with_spacy(text, skills, titles)

    print("\n===== RESUME PARSE RESULT =====")
    print("Top 500 Characters of Extracted Text:\n", text[:500], "\n")
    print("Skills Found:", found_skills)
    print("Job Titles Found:", found_titles)

    return {
        "text": text,
        "skills": found_skills,
        "titles": found_titles
    }

if __name__ == "__main__":
    file_path = r"C:\Users\velam\OneDrive\Documents\pavan's resume.docx"
    skills_folder = r"C:\amrita_uni\Projects\asmacs internship\recognition of skills and title\unique_job_skills_parts"
    titles_folder = r"C:\amrita_uni\Projects\asmacs internship\recognition of skills and title\unique_job_titles_parts"
    file_path2 = r"C:\amrita_uni\Projects\asmacs internship\recognition of skills and title\Imgdownload2.pdf"
    
    result1 = process_resume(file_path, skills_folder, titles_folder)
    result2 = process_resume(file_path2, skills_folder, titles_folder)


[INFO] Reading DOCX file...
[INFO] Loading skills and job titles...
[INFO] Extracting skills and titles using spaCy...

===== RESUME PARSE RESULT =====
Top 500 Characters of Extracted Text:
 Velamala Pavan Krishna

SUMMARY
Passionate computer science and AI student with a strong foundation in machine learning, data analysis, and model building . Eager to explore emerging technologies and apply creative solutions to real-world problems . Committed to continuous learning, collaboration, and making a meaningful impact through AI.

Phone:
+91 7569637875
Email:
velamalapavankrishna@gmail.com
Address: Visakhapatnam , 
Andhra Pradesh , India
    530046

Profile links

Linkedin: linkedin.co 

Skills Found: ['management', '2024', 'emerging', 'India', 'REDUCTION', 'Passionate', 'Skills\nTechnical', 'B', 'CONFERENCE', 'GPA', 'to', 'solutions', 'java', 'making', 'Certified', 'of', 'Interact', 'py', 'management system', 'Landmark', 'Thinking', 'data analysis', 'Large Language Models', 'Language', '

new gpt trail code (leave it)

In [1]:
import os
import spacy
from spacy.matcher import PhraseMatcher
from pdf2image import convert_from_path
import pytesseract
from docx import Document

# Set your poppler and tesseract paths
tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
poppler_path = r"C:\Release-24.08.0-0\poppler-24.08.0\Library\bin"
pytesseract.pytesseract.tesseract_cmd = tesseract_cmd

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Blocklists for filtering false positives
SKILL_BLOCKLIST = {"communication", "teamwork", "microsoft office", "ability", "work", "english"}
TITLE_BLOCKLIST = {"member", "participant", "student"}

# Context triggers for skills
SKILL_CONTEXT_TRIGGERS = ["experienced", "proficient", "skilled", "knowledge", "familiar"]
SKILL_PREPOSITIONS = ["in", "with", "on"]

# Common job title keywords
TITLE_KEYWORDS = ["developer", "engineer", "manager", "specialist", "consultant", "analyst"]

def extract_text_from_pdf(pdf_path):
    print("[INFO] Converting PDF pages to images...")
    images = convert_from_path(pdf_path, poppler_path=poppler_path)
    text = ""
    for i, img in enumerate(images):
        print(f"[INFO] Running OCR on page {i+1}")
        text += pytesseract.image_to_string(img)
    return text

def extract_text_from_docx(docx_path):
    print("[INFO] Reading DOCX file...")
    doc = Document(docx_path)
    return "\n".join([para.text for para in doc.paragraphs])

def load_keywords_from_folder(folder_path):
    keywords = set()
    for filename in os.listdir(folder_path):
        if filename.endswith(".txt") or filename.endswith(".csv"):
            with open(os.path.join(folder_path, filename), encoding='utf-8') as f:
                for line in f:
                    kw_clean = line.strip().lower()
                    if kw_clean:
                        keywords.add(kw_clean)
    return list(keywords)

def create_phrase_matcher(keywords, label):
    matcher = PhraseMatcher(nlp.vocab, attr="LOWER")
    patterns = [nlp.make_doc(text) for text in keywords]
    matcher.add(label, patterns)
    return matcher

def is_valid_skill(text):
    text_lower = text.lower()
    if any(blocked in text_lower for blocked in SKILL_BLOCKLIST):
        return False
    if len(text) < 2 or len(text) > 40:
        return False
    # Optionally allow digits for versions (e.g., Python 3)
    # if any(char.isdigit() for char in text):
    #     return False
    return True

def is_valid_title(text):
    text_lower = text.lower()
    if any(blocked in text_lower for blocked in TITLE_BLOCKLIST):
        return False
    if not any(keyword in text_lower for keyword in TITLE_KEYWORDS):
        return False
    if len(text) < 2 or len(text) > 60:
        return False
    return True

def find_contextual_skills(doc):
    skills = set()
    for sent in doc.sents:
        for token in sent:
            if token.lemma_.lower() in SKILL_CONTEXT_TRIGGERS:
                for child in token.children:
                    if child.dep_ == "prep" and child.text.lower() in SKILL_PREPOSITIONS:
                        for obj in child.children:
                            if obj.dep_ == "pobj" and is_valid_skill(obj.text):
                                skills.add(obj.text)
    return skills

def extract_keywords_with_spacy(text, skills_keywords, titles_keywords):
    doc = nlp(text)
    skills_matcher = create_phrase_matcher(skills_keywords, "SKILL")
    titles_matcher = create_phrase_matcher(titles_keywords, "TITLE")

    skills_found = set()
    titles_found = set()

    # PhraseMatcher results
    for match_id, start, end in skills_matcher(doc):
        span = doc[start:end]
        if is_valid_skill(span.text):
            skills_found.add(span.text)

    for match_id, start, end in titles_matcher(doc):
        span = doc[start:end]
        if is_valid_title(span.text):
            titles_found.add(span.text)

    # Contextual skill extraction
    skills_found.update(find_contextual_skills(doc))

    # NER results
    for ent in doc.ents:
        if ent.label_.lower() in {"skill", "job_title", "work_of_art", "org", "product"}:
            if any(keyword in ent.text.lower() for keyword in TITLE_KEYWORDS):
                if is_valid_title(ent.text):
                    titles_found.add(ent.text)
            else:
                if is_valid_skill(ent.text):
                    skills_found.add(ent.text)

    return sorted(skills_found), sorted(titles_found)

def process_resume(file_path, skills_folder, titles_folder):
    if file_path.endswith(".pdf"):
        text = extract_text_from_pdf(file_path)
    elif file_path.endswith(".docx"):
        text = extract_text_from_docx(file_path)
    else:
        raise ValueError("Unsupported file type. Please use PDF or DOCX.")

    print("[INFO] Loading skills and job titles...")
    skills = load_keywords_from_folder(skills_folder)
    titles = load_keywords_from_folder(titles_folder)

    print("[INFO] Extracting skills and titles using spaCy...")
    found_skills, found_titles = extract_keywords_with_spacy(text, skills, titles)

    print("\n===== RESUME PARSE RESULT =====")
    print("Top 500 Characters of Extracted Text:\n", text[:500], "\n")
    print("Skills Found:", found_skills)
    print("Job Titles Found:", found_titles)

    return {
        "text": text,
        "skills": found_skills,
        "titles": found_titles
    }

# === EXAMPLE USAGE ===
if __name__ == "__main__":
    file_path = r"C:\Users\velam\OneDrive\Documents\pavan's resume.docx"
    skills_folder = r"C:\amrita_uni\Projects\asmacs internship\recognition of skills and title\skills"
    titles_folder = r"C:\amrita_uni\Projects\asmacs internship\recognition of skills and title\titles"
    file_path2 = r"C:\amrita_uni\Projects\asmacs internship\recognition of skills and title\Imgdownload2.pdf"
    
    result1 = process_resume(file_path, skills_folder, titles_folder)
    result2 = process_resume(file_path2, skills_folder, titles_folder)


[INFO] Reading DOCX file...
[INFO] Loading skills and job titles...
[INFO] Extracting skills and titles using spaCy...

===== RESUME PARSE RESULT =====
Top 500 Characters of Extracted Text:
 Velamala Pavan Krishna

SUMMARY
Passionate computer science and AI student with a strong foundation in machine learning, data analysis, and model building . Eager to explore emerging technologies and apply creative solutions to real-world problems . Committed to continuous learning, collaboration, and making a meaningful impact through AI.

Phone:
+91 7569637875
Email:
velamalapavankrishna@gmail.com
Address: Visakhapatnam , 
Andhra Pradesh , India
    530046

Profile links

Linkedin: linkedin.co 

Skills Found: ['AI', 'AI & Development', 'AI Club', 'AI Fundamentals', 'Adaptive learning', 'Algorithms', 'Andhra Pradesh', 'Artificial Intelligence', 'Azure', 'B-Tech', 'Bus', 'CERTIFICATIONS', 'CS', 'CSS', 'Chess', 'Collaboration', 'Computer Science', 'Critical Thinking', 'Data Science', 'Data Structure

In [1]:
from sentence_transformers import SentenceTransformer

# Download and load the model from Hugging Face
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2", token=False)


# Save the model to your desired local path
model.save(r"C:\amrita_uni\Projects\asmacs internship\model")


In [4]:
import os
from pdf2image import convert_from_path
import pytesseract
from docx import Document
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Set your poppler and tesseract paths
tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
poppler_path = r"C:\Release-24.08.0-0\poppler-24.08.0\Library\bin"
pytesseract.pytesseract.tesseract_cmd = tesseract_cmd

# Load Sentence Transformer model
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")


def extract_text_from_pdf(pdf_path):
    print("[INFO] Converting PDF pages to images...")
    images = convert_from_path(pdf_path, poppler_path=poppler_path)
    text = ""
    for i, img in enumerate(images):
        print(f"[INFO] Running OCR on page {i+1}")
        text += pytesseract.image_to_string(img)
    return text

def extract_text_from_docx(docx_path):
    print("[INFO] Reading DOCX file...")
    doc = Document(docx_path)
    return "\n".join([para.text for para in doc.paragraphs])

def load_keywords_from_folder(folder_path):
    keywords = set()
    for filename in os.listdir(folder_path):
        if filename.endswith(".txt") or filename.endswith(".csv"):
            with open(os.path.join(folder_path, filename), encoding='utf-8') as f:
                for line in f:
                    kw_clean = line.strip().lower()
                    if kw_clean:
                        keywords.add(kw_clean)
    return list(keywords)

def create_faiss_index(embeddings):
    dim = embeddings.shape[1]
    index = faiss.IndexFlatIP(dim)  # Use inner product for cosine similarity (with normalized vectors)
    faiss.normalize_L2(embeddings)
    index.add(embeddings)
    return index

def extract_keywords_with_faiss(text, skills_keywords, titles_keywords):
    # Embed the input text as a single chunk
    text_embedding = model.encode([text], normalize_embeddings=True)

    # Embed keywords
    skills_embeddings = model.encode(skills_keywords, normalize_embeddings=True)
    titles_embeddings = model.encode(titles_keywords, normalize_embeddings=True)

    # Create FAISS indices
    skills_index = create_faiss_index(skills_embeddings)
    titles_index = create_faiss_index(titles_embeddings)

    # Retrieve top matches (top 10)
    k = 10
    D_skills, I_skills = skills_index.search(text_embedding, k)
    D_titles, I_titles = titles_index.search(text_embedding, k)

    # Collect matched keywords with similarity scores (threshold > 0.5)
    matched_skills = []
    for idx, i in enumerate(I_skills[0]):
        sim = float(D_skills[0][idx])
        if sim > 0.5:
            matched_skills.append((skills_keywords[i], sim))

    matched_titles = []
    for idx, i in enumerate(I_titles[0]):
        sim = float(D_titles[0][idx])
        if sim > 0.5:
            matched_titles.append((titles_keywords[i], sim))

    # Remove duplicates (keep highest similarity)
    skills_dict = {}
    for skill, sim in matched_skills:
        if skill not in skills_dict or sim > skills_dict[skill]:
            skills_dict[skill] = sim
    titles_dict = {}
    for title, sim in matched_titles:
        if title not in titles_dict or sim > titles_dict[title]:
            titles_dict[title] = sim

    # Sort by similarity descending
    matched_skills = sorted(skills_dict.items(), key=lambda x: -x[1])
    matched_titles = sorted(titles_dict.items(), key=lambda x: -x[1])

    return matched_skills, matched_titles

def process_resume(file_path, skills_folder, titles_folder):
    if file_path.endswith(".pdf"):
        text = extract_text_from_pdf(file_path)
    elif file_path.endswith(".docx"):
        text = extract_text_from_docx(file_path)
    else:
        raise ValueError("Unsupported file type. Please use PDF or DOCX.")

    print("[INFO] Loading skills and job titles...")
    skills = load_keywords_from_folder(skills_folder)
    titles = load_keywords_from_folder(titles_folder)

    print("[INFO] Extracting skills and titles using FAISS and Sentence Transformer...")
    found_skills, found_titles = extract_keywords_with_faiss(text, skills, titles)

    print("\n===== RESUME PARSE RESULT =====")
    print("Top 500 Characters of Extracted Text:\n", text[:500], "\n")
    print("Skills Found (with similarity):")
    for skill, sim in found_skills:
        print(f"{skill}: {sim:.4f}")
    print("Job Titles Found (with similarity):")
    for title, sim in found_titles:
        print(f"{title}: {sim:.4f}")

    return {
        "text": text,
        "skills": found_skills,
        "titles": found_titles
    }

if __name__ == "__main__":
    file_path = r"C:\Users\velam\OneDrive\Documents\pavan's resume.docx"
    skills_folder = r"C:\amrita_uni\Projects\asmacs internship\recognition of skills and title\unique_job_skills_parts"
    titles_folder = r"C:\amrita_uni\Projects\asmacs internship\recognition of skills and title\unique_job_titles_parts"
    file_path2 = r"C:\amrita_uni\Projects\asmacs internship\recognition of skills and title\Imgdownload2.pdf"

    result1 = process_resume(file_path, skills_folder, titles_folder)
    result2 = process_resume(file_path2, skills_folder, titles_folder)


No sentence-transformers model found with name sentence-transformers/all-MiniLM-L6-v2. Creating a new one with mean pooling.


OSError: sentence-transformers/all-MiniLM-L6-v2 is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [2]:
import os
from pdf2image import convert_from_path
import pytesseract
from docx import Document
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import re
import spacy

# Set your poppler and tesseract paths
tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
poppler_path = r"C:\Release-24.08.0-0\poppler-24.08.0\Library\bin"
pytesseract.pytesseract.tesseract_cmd = tesseract_cmd

# Load spaCy small model for NER
nlp = spacy.load("en_core_web_sm")

# Load Sentence Transformer model from local path
model = SentenceTransformer(r"C:\amrita_uni\Projects\asmacs internship\model")

# Regex patterns
email_pattern = r"[\w\.-]+@[\w\.-]+"
phone_pattern = r"\+?\d[\d\s\-]{8,}"

def extract_text_from_pdf(pdf_path):
    print("[INFO] Converting PDF pages to images...")
    images = convert_from_path(pdf_path, poppler_path=poppler_path)
    text = ""
    for i, img in enumerate(images):
        print(f"[INFO] Running OCR on page {i+1}")
        text += pytesseract.image_to_string(img)
    return text

def extract_text_from_docx(docx_path):
    print("[INFO] Reading DOCX file...")
    doc = Document(docx_path)
    return "\n".join([para.text for para in doc.paragraphs])

def load_keywords_from_folder(folder_path):
    keywords = set()
    for filename in os.listdir(folder_path):
        if filename.endswith(".txt") or filename.endswith(".csv"):
            with open(os.path.join(folder_path, filename), encoding='utf-8') as f:
                for line in f:
                    kw_clean = line.strip().lower()
                    if kw_clean:
                        keywords.add(kw_clean)
    return list(keywords)

def create_faiss_index(embeddings):
    dim = embeddings.shape[1]
    index = faiss.IndexFlatIP(dim)
    faiss.normalize_L2(embeddings)
    index.add(embeddings)
    return index

def extract_keywords_with_faiss(text, skills_keywords, titles_keywords, k=10, threshold=0.5):
    # Embed the input text as a single chunk
    text_embedding = model.encode([text], normalize_embeddings=True)

    # Embed keywords
    skills_embeddings = model.encode(skills_keywords, normalize_embeddings=True)
    titles_embeddings = model.encode(titles_keywords, normalize_embeddings=True)

    # Create FAISS indices
    skills_index = create_faiss_index(skills_embeddings)
    titles_index = create_faiss_index(titles_embeddings)

    # Retrieve top matches
    D_skills, I_skills = skills_index.search(text_embedding, k)
    D_titles, I_titles = titles_index.search(text_embedding, k)

    # Collect matched keywords with similarity scores (threshold > 0.5)
    matched_skills = []
    for idx, i in enumerate(I_skills[0]):
        sim = float(D_skills[0][idx])
        if sim > threshold:
            matched_skills.append((skills_keywords[i], sim))

    matched_titles = []
    for idx, i in enumerate(I_titles[0]):
        sim = float(D_titles[0][idx])
        if sim > threshold:
            matched_titles.append((titles_keywords[i], sim))

    # Remove duplicates (keep highest similarity)
    skills_dict = {}
    for skill, sim in matched_skills:
        if skill not in skills_dict or sim > skills_dict[skill]:
            skills_dict[skill] = sim
    titles_dict = {}
    for title, sim in matched_titles:
        if title not in titles_dict or sim > titles_dict[title]:
            titles_dict[title] = sim

    # Sort by similarity descending
    matched_skills = sorted(skills_dict.items(), key=lambda x: -x[1])
    matched_titles = sorted(titles_dict.items(), key=lambda x: -x[1])

    return matched_skills, matched_titles

def extract_email(text):
    match = re.search(email_pattern, text)
    return (match.group(0), 9.0) if match else ("", 0.0)

def extract_phone(text):
    match = re.search(phone_pattern, text)
    return (match.group(0), 8.5) if match else ("", 0.0)

def extract_name(text):
    doc = nlp(text)
    for ent in doc.ents:
        if ent.label_ == "PERSON":
            parts = ent.text.split()
            if parts:
                return (parts[0], 9.0)
    return ("", 0.0)

def extract_lastname(text):
    doc = nlp(text)
    for ent in doc.ents:
        if ent.label_ == "PERSON":
            parts = ent.text.split()
            if parts:
                return (parts[-1], 9.0)
    return ("", 0.0)

def process_resume(file_path, skills_folder, titles_folder):
    # Extract text from file
    if file_path.endswith(".pdf"):
        text = extract_text_from_pdf(file_path)
    elif file_path.endswith(".docx"):
        text = extract_text_from_docx(file_path)
    else:
        raise ValueError("Unsupported file type. Please use PDF or DOCX.")

    print("[INFO] Loading skills and job titles...")
    skills = load_keywords_from_folder(skills_folder)
    titles = load_keywords_from_folder(titles_folder)

    print("[INFO] Extracting skills and titles using FAISS and Sentence Transformer...")
    found_skills, found_titles = extract_keywords_with_faiss(text, skills, titles)

    email, email_conf = extract_email(text)
    phone, phone_conf = extract_phone(text)
    name, name_conf = extract_name(text)
    lastname, lastname_conf = extract_lastname(text)

    # Build JSON output
    resume_json = {
        "resumeData": {
            "basicDetails": {
                "name": {"value": name, "confidence": name_conf},
                "lastname": {"value": lastname, "confidence": lastname_conf},
                "email": {"value": email, "confidence": email_conf},
                "phone": {"value": phone, "confidence": phone_conf},
                "address": {"value": "", "confidence": 0.0}
            },
            "workExperience": [],
            "education": [],
            "skills": [{"skill": {"value": s, "confidence": round(sim*10, 2)}} for s, sim in found_skills],
            "certifications": []
        },
        "summary": {"value": "", "confidence": 0.0},
        "htmlContent": "",
        "pdfText": text
    }

    print("\n===== RESUME PARSE RESULT =====")
    print("Top 500 Characters of Extracted Text:\n", text[:500], "\n")
    print("Skills Found (with confidence):")
    for skill, sim in found_skills:
        print(f"{skill}: {sim:.4f}")
    print("Job Titles Found (with confidence):")
    for title, sim in found_titles:
        print(f"{title}: {sim:.4f}")

    return resume_json

if __name__ == "__main__":
    file_path = r"C:\Users\velam\OneDrive\Documents\pavan's resume.docx"
    skills_folder = r"C:\amrita_uni\Projects\asmacs internship\recognition of skills and title\unique_job_skills_parts"
    titles_folder = r"C:\amrita_uni\Projects\asmacs internship\recognition of skills and title\unique_job_titles_parts"

    result = process_resume(file_path, skills_folder, titles_folder)
    print(result)


[INFO] Reading DOCX file...
[INFO] Loading skills and job titles...
[INFO] Extracting skills and titles using FAISS and Sentence Transformer...

===== RESUME PARSE RESULT =====
Top 500 Characters of Extracted Text:
 Velamala Pavan Krishna

SUMMARY
Passionate computer science and AI student with a strong foundation in machine learning, data analysis, and model building . Eager to explore emerging technologies and apply creative solutions to real-world problems . Committed to continuous learning, collaboration, and making a meaningful impact through AI.

Phone:
+91 7569637875
Email:
velamalapavankrishna@gmail.com
Address: Visakhapatnam , 
Andhra Pradesh , India
    530046

Profile links

Linkedin: linkedin.co 

Skills Found (with confidence):
ai health tech: 0.5893
ai compute experience: 0.5852
microsoft ai: 0.5743
azure ai: 0.5704
ai platform  (openai): 0.5597
ai developer: 0.5567
ai training tool: 0.5562
ai integration developer: 0.5551
ai software: 0.5546
ai compute: 0.5529
Job Titles

using ollama llama3 with prompt eng 

In [ ]:
import os
import requests
import json
from pdf2image import convert_from_path
import pytesseract
from docx import Document

# Set your poppler and tesseract paths
tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
poppler_path = r"C:\Release-24.08.0-0\poppler-24.08.0\Library\bin"
pytesseract.pytesseract.tesseract_cmd = tesseract_cmd

OLLAMA_HOST = "http://localhost:11434"
OLLAMA_MODEL = "llama3"

def extract_text_from_pdf(pdf_path):
    print("[INFO] Converting PDF pages to images...")
    images = convert_from_path(pdf_path, poppler_path=poppler_path)
    text = ""
    for i, img in enumerate(images):
        print(f"[INFO] Running OCR on page {i+1}")
        text += pytesseract.image_to_string(img)
    return text

def extract_text_from_docx(docx_path):
    print("[INFO] Reading DOCX file...")
    doc = Document(docx_path)
    return "\n".join([para.text for para in doc.paragraphs])

def ollama_parse_resume(text):
    # Highly explicit, structured prompt for best accuracy
    prompt = f"""
You are a professional resume parsing assistant. Extract the following fields from the resume text below, using only information explicitly present in the text. Do not guess or hallucinate. If a field is missing, use an empty string or empty list.

Return ONLY a valid JSON object with this structure:
{{
  "basicDetails": {{
    "name": "First name only, as found in text",
    "lastname": "Last name only, as found in text",
    "email": ["all email addresses found, as a list"],
    "phone": "phone number as found"
  }},
  "skills": [
    {{"skill": {{"value": "skill name", "confidence": 10.0}}}}
  ],
  "titles": [
    {{"title": {{"value": "job title", "confidence": 10.0}}}}
  ],
  "workExperience": [
    {{
      "company": "company name",
      "position": "job title",
      "duration": "duration or years"
    }}
  ],
  "education": [
    {{
      "degree": "degree name",
      "institution": "institution name",
      "year": "year or range"
    }}
  ],
  "certifications": ["certification names as list"]
}}

Resume Text:
{text}
"""
    try:
        response = requests.post(
            f"{OLLAMA_HOST}/api/generate",
            json={"model": OLLAMA_MODEL, "prompt": prompt, "stream": False},
            timeout=120
        )
        response.raise_for_status()
        result = response.json()
        # Extract JSON from response
        start = result['response'].find('{')
        end = result['response'].rfind('}') + 1
        parsed = json.loads(result['response'][start:end])
    except Exception as e:
        print(f"[ERROR] Ollama parsing failed: {e}")
        parsed = {"error": str(e), "raw_response": result.get('response', '')}
    return parsed

def process_resume(file_path):
    # Extract text from file
    if file_path.endswith(".pdf"):
        text = extract_text_from_pdf(file_path)
    elif file_path.endswith(".docx"):
        text = extract_text_from_docx(file_path)
    else:
        raise ValueError("Unsupported file type. Please use PDF or DOCX.")

    print("[INFO] Parsing resume with Ollama...")
    parsed_data = ollama_parse_resume(text)
    resume_json = {
        "resumeData": {
            "basicDetails": parsed_data.get("basicDetails", {}),
            "workExperience": parsed_data.get("workExperience", []),
            "education": parsed_data.get("education", []),
            "skills": parsed_data.get("skills", []),
            "titles": parsed_data.get("titles", []),
            "certifications": parsed_data.get("certifications", [])
        },
        "summary": {"value": "", "confidence": 0.0},
        "htmlContent": "",
        "pdfText": text
    }

    print("\n===== RESUME PARSE RESULT =====")
    print("Top 500 Characters of Extracted Text:\n", text[:500], "\n")
    print(json.dumps(resume_json, indent=2))
    return resume_json

if __name__ == "__main__":
    
    file_path2=r"C:\amrita_uni\Projects\asmacs internship\Imgdownload2.pdf"
    result2= process_resume(file_path2)
    file_path = r"C:\Users\velam\OneDrive\Documents\pavan's resume.docx"
    result = process_resume(file_path)
    


[INFO] Converting PDF pages to images...
[INFO] Running OCR on page 1
[INFO] Running OCR on page 2
[INFO] Running OCR on page 3
[INFO] Running OCR on page 4
[INFO] Parsing resume with Ollama...

===== RESUME PARSE RESULT =====
Top 500 Characters of Extracted Text:
 DIGVIJAYSINGHPARMAR

Permanent Address: 40/C, Sangam Nagar, Indore, M.P — 452006
Present Address:2195 NiralaKunj, Gas AthourityOf India Township
Dibiyapur, Auraiya, Uttar Pradesh, |ndia-206244

Emait:digv10sp@gmail.com, digvijay10sp@outlook.com
Mobile: +91-9662489663

in ee Jinkedin.com/in/digvijay-singh-parmar-aabb0953

CAREER OBJECTIVE

To utilize my ability and knowledge about Safety and Emergency control methods, including
the detection of risks to prevent life-threatening harms and damages  

{
  "resumeData": {
    "basicDetails": {
      "name": "Digvijay Singh Parmar",
      "lastname": "",
      "email": [
        "digv10sp@gmail.com",
        "digvijay10sp@outlook.com"
      ],
      "phone": "+91-9662489663"
    }

with skills and titles finetuning


In [2]:
import os
import csv
import json
import random

def load_keywords_from_folder(folder_path):
    keywords = set()
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if filename.endswith(".txt"):
            with open(file_path, encoding='utf-8') as f:
                for line in f:
                    kw_clean = line.strip().lower()
                    if kw_clean:
                        keywords.add(kw_clean)
        elif filename.endswith(".csv"):
            with open(file_path, encoding='utf-8') as f:
                reader = csv.reader(f)
                for row in reader:
                    for item in row:
                        kw_clean = item.strip().lower()
                        if kw_clean:
                            keywords.add(kw_clean)
    return list(keywords)

# Set your paths
skills_folder = r"C:\amrita_uni\Projects\asmacs internship\recognition of skills and title\unique_job_skills_parts"
titles_folder = r"C:\amrita_uni\Projects\asmacs internship\recognition of skills and title\unique_job_titles_parts"

skills = load_keywords_from_folder(skills_folder)
titles = load_keywords_from_folder(titles_folder)

# Create training data
examples = []
for skill in skills:
    examples.append({
        "prompt": f"Extract skills and titles: The candidate is skilled in {skill}.",
        "completion": json.dumps({"skills": [skill], "titles": []})
    })

for title in titles:
    examples.append({
        "prompt": f"Extract skills and titles: The candidate worked as a {title}.",
        "completion": json.dumps({"skills": [], "titles": [title]})
    })

# Add 10,000 mixed examples
for _ in range(10000):
    skill = random.choice(skills)
    title = random.choice(titles)
    examples.append({
        "prompt": f"Extract skills and titles: Worked as {title} with {skill} skills.",
        "completion": json.dumps({"skills": [skill], "titles": [title]})
    })

# Save training data
with open("finetune_data.jsonl", "w", encoding="utf-8") as f:
    for ex in examples:
        f.write(json.dumps(ex) + "\n")
